<a href="https://colab.research.google.com/github/JP2620/Mentoria-Analisis-y-Visualizacion/blob/main/Susana_TP1_Datos_faltantes_Y_nulos__Mentoria09.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# **Mentoría Ciencia de datos aplicada a la predicción de licencias médicas y comportamiento de los colaboradores**
Trabajaremos con datos obtenidos de una plataforma de salud ocupacional con el **objetivo de predecir la carga de licencias médicas**. Se propone **realizar un análisis temporal a diferentes escalas**, e identificar las variables asociadas que **expliquen el número de licencias y/o diagnósticos involucrados**.
## **Trabajo Práctico 1: Análisis y Visualización de Datos**

---

# Datos



In [ ]:
import io
import matplotlib
import matplotlib.pyplot as plt
import numpy
import pandas as pd
import seaborn as sns

sns.set_context('talk')

## Lectura del dataset


In [ ]:
url = 'https://raw.githubusercontent.com/MentoriaDiploDatos/Presentacion/main/data.csv'
df = pd.read_csv(url, sep=';')

### #1 **¿Cuáles son las columnas relevantes del dataset? **
Selecciono las columnas más relevantes en base al objetivo:


*   observar si existe un sobredimensionamiento en la cantidad de días solicitados en las licencias médicas
*   el análisis debe ser temporal por lo que son importantes las fechas para realizarlo.

Para lo cual, reviso la info general del DS (dataset) lo que me permite determinar el total de filas,
los tipos de datos de cada columna,
las columnas que no contienen dato (nAn)




In [ ]:
# Reviso la información gral. del dataset
df.info()

In [ ]:
df.columns


De la info recogida en el item 1 se observa que las fechas tienen tipo de dato "object" y lo modificaré para posteriormente realizar análisis de series de tiempo

In [ ]:
df['fecha_creacion'] = pd.to_datetime(df['fecha_creacion'])
df['fecha_inicio'] = pd.to_datetime(df['fecha_inicio'])
df['fecha_fin'] = pd.to_datetime(df['fecha_fin'])
df['fecha_nacimiento'] = pd.to_datetime(df['fecha_nacimiento'])
df.dtypes
df.info()

In [ ]:
# Construyo un dataset con las columnas relevantes
relevant_columns = ['fecha_creacion',
       'dias_solicitados', 'fecha_inicio', 'fecha_fin', 'diagnostico',
       'id_diagnostico', 'fecha_nacimiento', 'genero',
       'tipo_licencia', 'estado_civil', 'dias_aprobados', 'decision_medica',
       'id_auditor', 'motivo_rechazo', 'categoria_diagnostico']
dfrelev = df[relevant_columns]
dfrelev[:3]

### 2 *¿Qué tipo de variable es cada una?
Asegurar que tengan el tipo de datos adecuado. Por ejemplo, para el caso de las fechas usar librería datetime para su conversión. En el paso anterior se realizó la transformación del tipo de dato de las fechas


In [ ]:
dfrelev.dtypes
dfrelev.info()

In [ ]:
#ordeno el set de datos por fecha de creación
dfrelev.sort_values(by='fecha_creacion', ascending=True).head(7)

## **3 Exploración del DataSet**
Se revisa el DataSet para revisar si se detectan;}:
Valores faltantes: aquellos datos que no se encuentran presentes en una variable o columna determinada, ya sea porque no se registraron o porque se perdieron.
Valores nulos: incompletos o corruptos.

## **Hay que decidir que gráfico es el más adecuado en cada caso para la acción a realizar**


**id_diagnostico** tiene 310 valores 0 pero es un código válido OTITIS EXTERNA mientras que hay  157021 valores nAn que deben ser completados con ENCODING tomando como referencia la columna DIAGNOSTICO
560 **dias_solicitados** en 0 que no encuentro a priori algún patrón para asignarles valor mientras que en nAn son 6035 que corresponden a los **tipo_licencia 'Médica_por_Hospitalización'** da cierto indicio que se trata del procedimiento del sistema de alta de licencias y por lo tanto las completaría con ENCODING
**id_auditor** 102674 tienen valor 0 que en todos los casos la decisión_médica fue APROBAR y los **dias_aprobados = dias_solicitados**
por otra parte hay 3079 filas nAn donde sólo 1 tiene **dias_aprobados** y **decision_medica** como un indicio de que se trata de los casos que aún no han sido evaluados por un auditor medico
**categoria_diagnostico** 158928 nAn pero tienen un diagnóstico y podría completarse con ENCODING
**NO ENCUENTRO PATRON PARA motivo_rechazo**

In [1]:
#Detección de las variables con valor cero del dataset.
cols = dfrelev[dfrelev == 0].count(axis=0)
cols[cols > 0]

NameError: ignored

In [ ]:
#Detección de las variables con valor nAn del dataset.
missing_values_count = dfrelev.isna().sum()
missing_values_count

In [ ]:
# 3 Exploren si hay valores faltantes y/o nulos en el dataset.
import missingno as msno
len(dfrelev)
print(len(dfrelev.dias_solicitados))
print(len(dfrelev.tipo_licencia))

#dfhospi = dfrelev[['dias_solicitados', 'tipo_licencia']].copy()

msno.matrix(dfrelev ,figsize=(12, 6), fontsize=12, color=[0,0,0.2])

#print(dfhospi['dias_solicitados'].isnull().sum())

print(dfrelev['dias_solicitados'].isnull().sum())
print(dfrelev['tipo_licencia'].isnull().sum())
print(dfrelev['dias_solicitados'].describe() )
print(dfrelev['tipo_licencia'].describe() )

Se cuentan la cantidad de filas correspondientes a cada columna para tener un indicio de la cantidad de datos faltantes

In [ ]:
#dfrelev_recha = pd.DataFrame({
#    'TIPO LIC': ['MÃ©dica', 'MÃ©dica por HospitalizaciÃ³n'],
#    'DIAS SOLIC': [0]
#})
#freq = dfrelev_recha.groupby(['TIPO LIC']).count()
#print(freq)

In [ ]:
# 3 Exploren si hay valores faltantes y/o nulos en el dataset.
dfrelev.count()

In [ ]:
dfrelev.sort_values(by='fecha_creacion', ascending=True).head(7)

In [ ]:
plt.figure(figsize=(10, 6))
plt.boxplot(data= dfrelev[dfrelev.dias_aprobados <365] , x='dias_aprobados',)
plt.show()


In [ ]:
#postings.boxplot(column='dias_aprobados', by='tipo_licencia')

### Calculo la cantidad de días efectivamente licenciados ***REVISAR EL ALGORITMO ***

In [ ]:
#4 CALCULO LA CANTIDAD DE DÍAS DE LA LICENCIA TOMADA.
from datetime import datetime
def days_between(row):
    d1 = datetime.strptime('fecha_inicio', "%Y-%m-%d")
    d2 = datetime.strptime('fecha_fin', "%Y-%m-%d")
    return abs((d2 - d1).days)

#def calculate_dias_licefect(row):
#    fecha_inicio = pd.to_datetime(row['fecha_inicio'])
#    fecha_fin = pd.to_datetime(row['fecha_fin'])
#    return fecha_fin.year - fecha_inicio.year - ((fecha_fin.month, fecha_fin.day) < (fecha_inicio.month, fecha_inicio.day))

if 'fecha_inicio' !=  data_string or 'fecha_fin' != data_string:
    dfrelev['dias_lic_efectiva'] = dfrelev.apply(days_between, axis=1)
dfrelev.dtypes

dfrelev.info()

In [ ]:
#Para las columnas numéricas de días_solicitados y días_aprobados verificamos
#si tienen valores en cero
#Si el tipo de licencia es Hospitalaria la cantidad de días_solicitados será 0 hasta determinar el tiempo de internación
#licencias = dfrelev.value_counts()
dnosolicita = dfrelev['dias_solicitados'].value_counts()[0]
khospital = dfrelev['tipo_licencia'].value_counts()['Médica por Hospitalización']

#print(licencias, ' ', dnosolicita,' ', khospital)

print(dnosolicita,' ',khospital)


In [ ]:
# de los 427003 licencias aprobadas, hay 22487 que les otorgaron 0 día de licencia
daprob = dfrelev['dias_aprobados'].value_counts()[>0]

## #4 CALCULO LA EDAD
A partir de la fecha de nacimiento calcular la edad del colaborador que solicita la licencia.

In [ ]:
#4 CALCULO LA EDAD A partir de la fecha de nacimiento calcular la edad del colaborador que solicita la licencia.
def calculate_age(row):
    fecha_nacimiento = pd.to_datetime(row['fecha_nacimiento'])
    fecha_inicio = pd.to_datetime(row['fecha_inicio'])
    return fecha_inicio.year - fecha_nacimiento.year - ((fecha_inicio.month, fecha_inicio.day) < (fecha_nacimiento.month, fecha_nacimiento.day))

dfrelev['edad'] = dfrelev.apply(calculate_age, axis=1)
dfrelev.dtypes


In [ ]:
dfrelev[:3]


#6 Realicen un análisis estadístico de cada una de las variables numéricas:
Cantidad de datos, mínimo, máximo, media, mediana, varianza, desviación estándar, cuartil 1, cuartil 3, rango intercuartílico.
    6.A) Elijan una o dos variables categóricas, repetir este análisis y sacar conclusiones.
    6.B) ¿El tiempo es una variable influyente en las distribuciones de estas variables? Ayudarse de gráficos para contestar esta pregunta.

In [ ]:
dfrelev.shape

In [ ]:
dfrelev.describe()

In [ ]:
# ESTADÍSTICAS DE DATOS NUMÉRICOS **DIAS SOLICITADOS**


import numpy as np



print(dfrelev['dias_solicitados'].describe() )
# dfrelev.apply(np.nanvar, axis=0) FALTA CALCULAR LA VARIANZA..

In [ ]:
# ESTADÍSTICAS DE DATOS NUMÉRICOS **DIAS APROBADOS**
print(dfrelev['dias_aprobados'].describe() )
# dfrelev.apply(np.nanvar, axis=0) FALTA CALCULAR LA VARIANZA..

In [ ]:
# ESTADÍSTICAS DE DATOS NUMÉRICOS **DIAS LICENCIA EFECTIVA**
print(dfrelev['dias_lic_efectiva'].describe() )
# dfrelev.apply(np.nanvar, axis=0) FALTA CALCULAR LA VARIANZA..

In [ ]:
# ESTADÍSTICAS DE DATOS NUMÉRICOS **EDAD**
print(dfrelev['edad'].describe() )
# dfrelev.apply(np.nanvar, axis=0) FALTA CALCULAR LA VARIANZA..